# Exercise 1

In [ ]:
#using Pkg
#Pkg.add("StatsBase");

## 1. Generating exponentially distributed random variables

Let us simulate $10^5$ realizations of a random variable $X$ that follows an Exponential distribution of parameter $\lambda = 10$. 

- with the uniform distribution

In [ ]:
λ = 10
g = y -> ####                   # write what your function should return here
u = rand(Integer(####))         # draw 10^5 realizations from a uniform distribution with rand()
g_eval = g.(u);                 # evaluate your function g at those realizations

In [ ]:
using Plots
histogram(g_eval, normalize = true, xlim = (0, 1), label = "from Uniform")

- with the exponential distribution

[1]https://github.com/JuliaStats/Distributions.jl/tree/ebeab79ff28f144506f6aa51b284b67486283ba0/src/univariate/continuous
[2]https://github.com/JuliaLang/julia/blob/master/stdlib/Random/src/normal.jl

In [ ]:
using Distributions
pdf_evaluation = x -> pdf.(Exponential(1/λ), x) # write what your function should return here
plot!(0:0.01:2.0, pdf_evaluation, 
    linewidth = 3.0, 
    label = "from Exponential")

## 2. Gillespie algorithm (naive implementation)

In [ ]:
κ = ####        # rate constant
x₀ = ####        # initial count of molecules
Δt = 0.001;    

In [ ]:
gillespie_alg_naive = function(x₀, κ, Δt, stoptime)
   

    tt = zeros(Integer(round(stoptime/Δt))+1)
    xx = Integer.(zeros(length(tt)))
   
    t = 0
    k = 0
    
    xx[1] = ####                      # initial count
    tt[1] = ####                      # initial time
    
    k = 1
    while (t < stoptime)
        x = xx[k]
        
        ####                          # steps 1 - 3

        t = k*Δt
        tt[k+1] = t
        k = k + 1 
    end
    
    return tt, xx
end;

In [ ]:
tt, xx = gillespie_alg_naive(x₀, κ, Δt, 30);

In [ ]:
plot(tt, xx, linetype=:steppre, xlabel = "time (sec)", ylabel = "# molecules of A", legend=false)

## 3. Gillespie algorithm

### Preliminary steps

Let us consider the reactions

$A \xrightarrow{\kappa_1} \emptyset$

$\emptyset \xrightarrow{\kappa_2}A $

\begin{align}
(\text{Reactants})\,\overline{S} = \begin{bmatrix}
n_{A,\,r_1}^{(\text{reac})}\\
n_{A,\,r_2}^{(\text{reac})}\\
\end{bmatrix}_{\text{#reactions }\times \text{ #species}} &= \begin{bmatrix}
1\\
0\\
\end{bmatrix}\\
\hspace{2cm}
(\text{Products})\,\underline{S} = \begin{bmatrix}
n_{A,\,r_1}^{(\text{prod})}\\
n_{A,\,r_2}^{(\text{prod})}\\
\end{bmatrix}_{\text{#reactions }\times \text{ #species}} &= \begin{bmatrix}
0\\
1\\
\end{bmatrix}
\end{align}

\begin{align}
(\text{Stoichiometric Matrix})\, S = (\underline{S} -\overline{S})^\top = \begin{bmatrix}
-1 & 1
\end{bmatrix}
\end{align}



In [ ]:
# Number of reactions
Nᵣ = 2

# Compute the stoichiometric matrix 
Products = [0; 1]
Reactants = [1; 0]
SM = (Products - Reactants)'

(step 1) Compute the propensity function $\nu_{n_r}(t)$ for each of the $N_r$ reactions. Then compute the total propensity  $\alpha(t)$.

In [ ]:
# Compute the propensity function for each of the reactions

V = 0.001                                       # Volume 
ν₁ =  function(κ₁, x) return #### end            # Propensity function for degradation
ν₂ = function(κ₂, x) return #### end             # Propensity function for production

In [ ]:
# Compute the total propensity
propensities = function(k, x, ν, Nᵣ)
    ν_eval = zeros(Nᵣ)
    for r in 1:Nᵣ
        ν_eval[r] = ν[r](k[r], x) 
    end
    return ν_eval
end

In [ ]:
# Example : 
k = [0.1, 1000]
x = zeros(1)
ν = [ν₁, ν₂]

ν_eval = ####                                  # Evaluate propensity functions 
α = sum(ν_eval)

(step 2) Compute the time when the first chemical reaction takes place as $\tau$, where $\tau$ is sampled from an exponential distribution with parameter $\alpha(t)$

In [ ]:
τ = ####                                       

(step 3) Find out which reaction occurs at time $\tau$

In [ ]:
using StatsBase
index_j = StatsBase.sample(1:2, Weights(ν_eval./α));
println("Reaction n." .* string(index_j) .* " takes place!")

In [ ]:
println("Update number of molecules A with the stoichiometric vector: " .* string(SM[:, index_j]))

### Gillespie algorithm (main algorithm)

Combine all these steps in a loop.

In [ ]:
gillespie_alg = function(SM, κ, ν, x₀, stoptime)
    t = 0.0
    x = x₀
    
    Nₛ, Nᵣ = size(SM)
    
    tt = [t]
    xx = copy(x)     
    
    while (t <= stoptime)
        
        # step 1
        ν_eval = ####             
        α = #### 

        # step 2
        τ = ####                                   
        t += τ

        # step 3
        index_j = StatsBase.sample(1:Nᵣ, Weights(ν_eval./α))
        
        x = x + ####
        append!(tt, t)
        xx = cat(xx, x, dims=2)
    end
    
    return tt, xx'
end

### (a) Stochastic simulation of degradation

In [ ]:
# Number of reactions Nᵣ = 1
# Compute the stoichiometric matrix 
Products = ####
Reactants = ####
SM = (Products-Reactants)'

In [ ]:
# Compute the propensity function for each of the reactions
V = 0.001
ν₁ =  function(κ₁, x) return #### end

1 realization

In [ ]:
tt, xx = gillespie_alg(SM, [0.1], [ν₁], [20], 100.0)
plot(tt[1:end-1], xx[1:end-1], linetype=:steppre, xlabel="time (sec)", ylabel ="# molecules of A")

10 realizations

In [ ]:
Nₛ = 20
p = plot(legend = false, xlabel = "time (sec)", ylabel = "# molecules of A")
for nₛ in 1:Nₛ    
    tt, xx = #### 
    plot!(tt[1:end-1], xx[1:end-1], linetype=:steppre)
end
p

### (b) stochastic simulation of production and degradation

In [ ]:
# Number of reactions Nᵣ = 2
# Compute the stoichiometric matrix 
Products = ####
Reactants = #### 
SM = (Products-Reactants)'

In [ ]:
# Compute the propensity function for each of the reactions
V = 0.001
ν₁ =  function(κ₁, x) return #### end
ν₂ = function(κ₂, x) return #### end 

1 realization

In [ ]:
κ₁ = 0.1
κ₂ = 1000.0

tt, xx = ####
plot(tt, xx, linetype=:steppre, label = "1 realization of Production + Degradation", legend=:bottomright)

10 realizations

In [ ]:
Nₛ = 10
p = plot(legend=false, xlabel="time (sec)", ylabel ="# molecules of A")
for nₛ in 1:Nₛ    
    tt, xx = ####
    plot!(tt[1:end-1], xx[1:end-1], linetype=:steppre)
end
p

Evolution of the stochastic mean $M(t)$

In [ ]:
M = t -> ####
plot!(tt[1:end-1], M, linestyle = :dash, colour = :black, linewidth = 3)

### (c) Steady-state version of the chemical master equation

Number of molecules at $t=100$sec, from $10^6$ realizations

In [ ]:
N = ####    #number of realisations
n_upperbound = 24
p = zeros(n_upperbound+1)   #vector of probabilities


for n in 1:N   
    tt, xx = ####
    A = xx[end]    
    if (A < n_upperbound-0.5) 
        p[xx[end]+1] += 1
    end
end

p /= N;

In [ ]:
bar(collect(0:24), p, 
    colour = :lightgrey,
    label = "Gillespie SSA",
    xlabel = "number of molecules",
    ylabel = "stationary distribution")

In [ ]:
compute_ϕ = function(n_upperbound, κ₁, κ₂)
    
    ϕ = zeros(n_upperbound+1)

    ϕ[1] =  1.0
    ϕ[2] = ####

    for n in 1:n_upperbound-1
       ϕ[n+2] = #### 
    end
    return ϕ./(sum(ϕ))
end

In [ ]:
ϕ = compute_ϕ(n_upperbound, 0.1, 1.0);

In [ ]:
plot!(collect(0:n_upperbound), ϕ,
    linewidth = 3,
    colour = :red,
    label = "master equation")

# Homework

## Problem 3

State variable

$X(t) = (N_G(t),N_M(t),N_P(t))$

Reactions

$\emptyset \xrightarrow{\kappa_1}M$

$M \xrightarrow{\kappa_2}M+P $

$M \xrightarrow{d_M}\emptyset $

$P \xrightarrow{d_P}\emptyset $



In [ ]:
# Number of reactions
Nᵣ = ####

# Compute the stoichiometric matrix 
Products = ####
Reactants = ####
SM = (Products-Reactants)'

Compute the propensity function for each of the reactions

In [ ]:
V = 1
ν₁ = function(κ₁, x) return #### end
ν₂ = function(κ₂, x) return #### end 
ν₃ = function(κ₃, x) return #### end 
ν₄ = function(κ₄, x) return #### end 
ν = [ν₁, ν₂, ν₃, ν₄]

In [ ]:
κ = ####;
x₀ = ####;

In [ ]:
tt, xx = gillespie_alg(SM, κ, ν, x₀, 8.0)

In [ ]:
plot(tt, xx, 
    linetype=:steppre, 
    labels = ["# of genes" "# of mRNA molecules" "# of proteins"], 
    xlabel = "Time",
    ylabel = "Counts",
    legend=:topleft)

(b) Protein dimerization

State variable

$X(t) = (N_G(t),N_M(t),N_P(t), N_D(t))$

Reactions

$\emptyset \xrightarrow{\kappa_1}M$

$M \xrightarrow{\kappa_2}M+P $

$M \xrightarrow{d_M}\emptyset $

$P \xrightarrow{d_P}\emptyset $

$2P \xrightarrow{\kappa_3}D $

$D \xrightarrow{d_d}\emptyset $


In [ ]:
# Number of reactions
Nᵣ = 6

# Compute the stoichiometric matrix 
Products = ####
Reactants = ####
SM = (Products-Reactants)';

In [ ]:
ν₅ = function(κ₅, x) return #### end 
ν₆ = function(κ₆, x) return #### end 
ν = [ν₁, ν₂, ν₃, ν₄, ν₅, ν₆]

κ = ####
x₀ = ####
tt, xx = gillespie_alg(SM, κ, ν, x₀, 20.0)
plot(tt, xx, 
    linetype = :steppre, 
    labels = ["# of genes" "# of mRNA molecules" "# of proteins" "# of dimers"], 
    xlabel = "Time",
    ylabel = "Counts",
    legend = :topleft)